## `In some places code can be simplified/removed/replaced`

### Importing all necessary packages and modules

In [1]:
import pandas as pd
import numpy as np
from datasets import load_metric
from datasets import Dataset
from transformers import (AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer)

### Loading and splitting data

In [2]:
df = pd.read_csv("../data/raw/filtered.tsv", sep="\t")
toxic = df[df['ref_tox'] > df['trn_tox']].iloc[:, 1:]   # Choosing the partition where the toxicity score of the reference is bigger than the one of its translation
toxic.head()

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
5,I'm not gonna have a child... ...with the same...,I'm not going to breed kids with a genetic dis...,0.703185,0.206522,0.950956,0.035846
6,"They're all laughing at us, so we'll kick your...",they're laughing at us. We'll show you.,0.618866,0.230769,0.999492,0.000131
7,Maine was very short on black people back then.,there wasn't much black in Maine then.,0.720482,0.187500,0.963680,0.148710
11,"So now their spirits are cursed, walking back ...","their souls are cursed, they guard the paths, ...",0.755883,0.013245,0.842509,0.143992
13,"Come on, Cal, leave that shit alone.","come on, Cal, put it down.",0.660481,0.270270,0.999637,0.000279


### Sampling an example 

In [3]:
print(toxic['reference'].values[100])
print(toxic['translation'].values[100])

If you don't stop laughing, you'll end up dead... just like your idiot hyena cousins!
if you don't stop laughing, you come as your cousins of hyenas.


### Taking only a part of data with references & their translations

In [4]:
sins = toxic[["reference", 'translation']]

### Here it is possible to configure the partition of data that will be taken from all samples for traing(was useful for "fast check" of the training process)

In [8]:
part_factor = sins.shape[0]
# part_factor = 1000

### Calculating the maximum lenght of the toxic sentence for model generation 

In [9]:
lens_r = sins[:part_factor]['reference'].str.lower().str.split().apply(lambda x: len(x))
lens_t = sins[:part_factor]['translation'].str.lower().str.split().apply(lambda x: len(x))
max_len = np.max([lens_r.max(), lens_t.max()])
print(max_len)

### Creating a dataset to use .map() method for preparing data for the model training

In [12]:
dataset = Dataset.from_pandas(sins[:part_factor]).train_test_split(test_size=0.2)   # Create dataset and split it on "train" and "test" parts

### Load model and tokenizer, tokenizing data, creating model and data collator objects

In [13]:
checkpoint = "t5-small" # Model to load - "T5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)   # Creating tokenizer

# Function for preprocessing sentences
def preprocess_function(data):
    inputs = [sent.lower().split() for sent in data["reference"]]
    targets = [sent.lower().split() for sent in data["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_len, truncation=True, is_split_into_words=True)

    return model_inputs

# Tokenizing the data in the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Creating a model's object
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Creating a data collator's object
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

Map:   0%|          | 0/255313 [00:00<?, ? examples/s]

Map:   0%|          | 0/63829 [00:00<?, ? examples/s]

### Functions for calculating BLEU score metric

In [15]:
metric = load_metric("sacrebleu")

# Function for preprocessing data for calculating metrics
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# Main function for calculating metrics: BLEU score and average lenght of generated sentences
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # Replacing -100 tokens(if any) on pad tokens of the tokenizer
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) # Decoding tokens to words

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)   # Replacing -100 tokens(if any) on pad tokens of the tokenizer
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # Decoding tokens to words
    
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) # Prepare sentences for calculating metrics

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)   # Calculating BLEU score for each sentence
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]  
    result["gen_len"] = np.mean(prediction_lens)    # Calculating average lenght of generated sentences 
    result = {k: round(v, 4) for k, v in result.items()}
    return result

C:\Users\Ario\AppData\Local\Temp\ipykernel_27328\3858898384.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


### Choosing training hyperparameters and configuring training process with consequent launching it

In [17]:
new_model = '../models/T5-small_2e-5_32_10_MaxTokenLength' # Path for models checkpoints

# All training arguments/hyperparameters
training_args = Seq2SeqTrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    generation_max_length = max_len + 5,
    weight_decay=1e-6,
)

# Trainer object creation
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Starting training
trainer.train()

  0%|          | 0/79790 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.0181, 'learning_rate': 1.987542298533651e-05, 'epoch': 0.06}
{'loss': 1.8386, 'learning_rate': 1.975009399674145e-05, 'epoch': 0.13}
{'loss': 1.7833, 'learning_rate': 1.9624765008146387e-05, 'epoch': 0.19}
{'loss': 1.7439, 'learning_rate': 1.9499436019551323e-05, 'epoch': 0.25}
{'loss': 1.7236, 'learning_rate': 1.9374107030956263e-05, 'epoch': 0.31}
{'loss': 1.7051, 'learning_rate': 1.92487780423612e-05, 'epoch': 0.38}
{'loss': 1.7026, 'learning_rate': 1.912369971174333e-05, 'epoch': 0.44}
{'loss': 1.6885, 'learning_rate': 1.8998370723148265e-05, 'epoch': 0.5}
{'loss': 1.6686, 'learning_rate': 1.8873041734553205e-05, 'epoch': 0.56}
{'loss': 1.6598, 'learning_rate': 1.8747712745958142e-05, 'epoch': 0.63}
{'loss': 1.6543, 'learning_rate': 1.862238375736308e-05, 'epoch': 0.69}
{'loss': 1.6467, 'learning_rate': 1.8497054768768018e-05, 'epoch': 0.75}
{'loss': 1.6412, 'learning_rate': 1.8371725780172955e-05, 'epoch': 0.81}
{'loss': 1.6379, 'learning_rate': 1.8246396791577894e-05, 

  0%|          | 0/1995 [00:00<?, ?it/s]

{'eval_loss': 1.493385672569275, 'eval_bleu': 28.7427, 'eval_gen_len': 16.3043, 'eval_runtime': 2547.2054, 'eval_samples_per_second': 25.058, 'eval_steps_per_second': 0.783, 'epoch': 1.0}
{'loss': 1.6316, 'learning_rate': 1.7995738814387767e-05, 'epoch': 1.0}
{'loss': 1.6065, 'learning_rate': 1.7870660483769897e-05, 'epoch': 1.07}
{'loss': 1.6049, 'learning_rate': 1.7745331495174837e-05, 'epoch': 1.13}
{'loss': 1.6093, 'learning_rate': 1.7620002506579773e-05, 'epoch': 1.19}
{'loss': 1.6169, 'learning_rate': 1.749467351798471e-05, 'epoch': 1.25}
{'loss': 1.6062, 'learning_rate': 1.736934452938965e-05, 'epoch': 1.32}
{'loss': 1.606, 'learning_rate': 1.7244015540794586e-05, 'epoch': 1.38}
{'loss': 1.5861, 'learning_rate': 1.7118937210176715e-05, 'epoch': 1.44}
{'loss': 1.5889, 'learning_rate': 1.6993608221581652e-05, 'epoch': 1.5}
{'loss': 1.5837, 'learning_rate': 1.6868279232986592e-05, 'epoch': 1.57}
{'loss': 1.5847, 'learning_rate': 1.6742950244391528e-05, 'epoch': 1.63}
{'loss': 1.575

...

  0%|          | 0/1995 [00:00<?, ?it/s]

{'eval_loss': 1.377905011177063, 'eval_bleu': 30.2792, 'eval_gen_len': 16.194, 'eval_runtime': 2716.4888, 'eval_samples_per_second': 23.497, 'eval_steps_per_second': 0.734, 'epoch': 9.0}
{'loss': 1.468, 'learning_rate': 1.9593934076952e-06, 'epoch': 9.02}
{'loss': 1.4739, 'learning_rate': 1.834064419100138e-06, 'epoch': 9.09}
{'loss': 1.4561, 'learning_rate': 1.708735430505076e-06, 'epoch': 9.15}
{'loss': 1.4717, 'learning_rate': 1.583406441910014e-06, 'epoch': 9.21}
{'loss': 1.4597, 'learning_rate': 1.458077453314952e-06, 'epoch': 9.27}
{'loss': 1.4724, 'learning_rate': 1.3329991226970797e-06, 'epoch': 9.34}
{'loss': 1.4575, 'learning_rate': 1.207920792079208e-06, 'epoch': 9.4}
{'loss': 1.4627, 'learning_rate': 1.0825918034841459e-06, 'epoch': 9.46}
{'loss': 1.4617, 'learning_rate': 9.572628148890838e-07, 'epoch': 9.53}
{'loss': 1.4635, 'learning_rate': 8.319338262940219e-07, 'epoch': 9.59}
{'loss': 1.4666, 'learning_rate': 7.066048376989598e-07, 'epoch': 9.65}
{'loss': 1.4647, 'learn

  0%|          | 0/1995 [00:00<?, ?it/s]

{'eval_loss': 1.3774222135543823, 'eval_bleu': 30.3103, 'eval_gen_len': 16.216, 'eval_runtime': 2770.9514, 'eval_samples_per_second': 23.035, 'eval_steps_per_second': 0.72, 'epoch': 10.0}
{'train_runtime': 44320.725, 'train_samples_per_second': 57.606, 'train_steps_per_second': 1.8, 'train_loss': 1.5262058627621873, 'epoch': 10.0}


TrainOutput(global_step=79790, training_loss=1.5262058627621873, metrics={'train_runtime': 44320.725, 'train_samples_per_second': 57.606, 'train_steps_per_second': 1.8, 'train_loss': 1.5262058627621873, 'epoch': 10.0})

### Load model checkpoint and generating detoxified sentence
    I hope that there is no need to comment anything

In [18]:
test_model = AutoModelForSeq2SeqLM.from_pretrained("../models/T5-small_detox/final_checkpoint/")

In [91]:
input_ids = tokenizer("Are we going to bring these self-pissing dogs here?".lower().split(),
                      max_length=max_len, truncation=True, is_split_into_words=True, return_tensors="pt").input_ids
outputs = test_model.generate(input_ids, max_new_tokens=max_len + 5)

In [92]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

are we going to bring these dogs here?
